In [368]:
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error



In [369]:
train_data = pd.read_csv('/Users/rade_dragosavac/PycharmProjects/SendyLogisticsChallenge /Data/Train.csv')
riders_data = pd.read_csv('/Users/rade_dragosavac/PycharmProjects/SendyLogisticsChallenge /Data/Riders.csv')
test_data = pd.read_csv('/Users/rade_dragosavac/PycharmProjects/SendyLogisticsChallenge /Data/Test.csv')


In [370]:
def time_from_midnight_in_seconds(data, column):
    data[column] = pd.to_datetime(data[column])
    return (data[column] - pd.to_datetime(pd.to_datetime('today').date())).astype('timedelta64[s]')


def delta_time(dataset, higher_time, lower_time):
    return dataset[higher_time] - dataset[lower_time]

def time_to_day_part(time):
    hours = time/3600
    if hours < 6:
        return ('Night')
    if hours < 12:
        return ('Morning')
    if hours < 18:
        return ('Afternoon')
    else:
        return ('Evening')
    
def calculate_bearing(lat1, lng1, lat2, lng2):
        lat1 = np.deg2rad(lat1)
        lat2 = np.deg2rad(lat2)
        diffLong = np.deg2rad(lng2 - lng1)
        x = np.sin(diffLong) * np.cos(lat2)
        y = np.cos(lat1) * np.sin(lat2) - (np.sin(lat1)
                                     * np.cos(lat2) * np.cos(diffLong))
        initial_bearing = np.arctan2(x, y)
        # Now we have the initial bearing but math.atan2 return values
        # from -180° to + 180° which is not what we want for a compass bearing
        # The solution is to normalize the initial bearing as shown below
        initial_bearing = np.rad2deg(initial_bearing)
        compass_bearing = (initial_bearing+360) % 360
        return compass_bearing    


In [371]:
# converting time in seconds from midnight
train_data['Placement - Time'] = time_from_midnight_in_seconds(train_data, 'Placement - Time')
train_data['Confirmation - Time'] = time_from_midnight_in_seconds(train_data, 'Confirmation - Time')
train_data['Pickup - Time'] = time_from_midnight_in_seconds(train_data, 'Pickup - Time')
train_data['Arrival at Pickup - Time'] = time_from_midnight_in_seconds(train_data, 'Arrival at Pickup - Time')
train_data['Arrival at Destination - Time'] = time_from_midnight_in_seconds(train_data, 'Arrival at Destination - Time')

# converting time in seconds from midnight for test data
test_data['Placement - Time'] = time_from_midnight_in_seconds(test_data, 'Placement - Time')
test_data['Confirmation - Time'] = time_from_midnight_in_seconds(test_data, 'Confirmation - Time')
test_data['Pickup - Time'] = time_from_midnight_in_seconds(test_data, 'Pickup - Time')
test_data['Arrival at Pickup - Time'] = time_from_midnight_in_seconds(test_data, 'Arrival at Pickup - Time')


In [276]:
# calculating delta_time for train data
delta_confirm_place_train = delta_time(train_data, 'Confirmation - Time', 'Placement - Time')
delta_pick_arr_confirm_train = delta_time(train_data, 'Arrival at Pickup - Time', 'Confirmation - Time')
delta_pickup_confirm_train = delta_time(train_data, 'Pickup - Time', 'Arrival at Pickup - Time')
delta_arrival_pickup_train = delta_time(train_data, 'Arrival at Destination - Time', 'Pickup - Time')
delta_placement_arrival_train = delta_time(train_data, 'Arrival at Destination - Time', 'Placement - Time')

delta_confirm_place_test = delta_time(test_data, 'Confirmation - Time', 'Placement - Time')
delta_pick_arr_confirm_test = delta_time(test_data, 'Arrival at Pickup - Time', 'Confirmation - Time')
delta_pickup_confirm_test = delta_time(test_data, 'Pickup - Time', 'Arrival at Pickup - Time')


In [372]:
train_with_rider_info = train_data.merge(riders_data, on='Rider Id')
test_with_rider_info = test_data.merge(riders_data, on='Rider Id')


In [373]:
train_with_rider_info = train_with_rider_info[train_with_rider_info['Placement - Weekday (Mo = 1)'] == train_with_rider_info['Confirmation - Weekday (Mo = 1)']]
train_with_rider_info = train_with_rider_info[train_with_rider_info['Placement - Day of Month'] == train_with_rider_info['Confirmation - Day of Month']]


In [374]:
train_with_rider_info['Temperature'].fillna(train_with_rider_info['Temperature'].mean(), inplace=True)
test_with_rider_info['Temperature'].fillna(test_with_rider_info['Temperature'].mean(), inplace=True)


In [375]:
# label encoding of personal/business column for train data
labelencoder_personal_business = LabelEncoder()
train_with_rider_info['Personal or Business'] = labelencoder_personal_business.fit_transform(train_with_rider_info['Personal or Business'])

# label encoding of personal/business column for test data
labelencoder_personal_business = LabelEncoder()
test_with_rider_info['Personal or Business'] = labelencoder_personal_business.fit_transform(test_with_rider_info['Personal or Business'])


In [376]:
# one hot encoding of the train_data['Platform Type'] column
train_with_rider_info['Platform Type'] = train_with_rider_info['Platform Type'].astype('category')
train_with_rider_info = pd.concat([train_with_rider_info.drop(columns=['Platform Type']), pd.get_dummies(train_with_rider_info['Platform Type'])], axis=1)



In [377]:
# one hot encoding of the test_data['Platform Type'] column
test_with_rider_info['Platform Type'] = test_with_rider_info['Platform Type'].astype('category')
test_with_rider_info = pd.concat([test_with_rider_info.drop(columns=['Platform Type']), pd.get_dummies(test_with_rider_info['Platform Type'])], axis=1)


In [378]:
bearing_test = calculate_bearing(test_with_rider_info['Pickup Lat'], test_with_rider_info['Pickup Long'],
                            test_with_rider_info['Destination Lat'], test_with_rider_info['Destination Long'])

bearing_train = calculate_bearing(train_with_rider_info['Pickup Lat'], train_with_rider_info['Pickup Long'],
                            train_with_rider_info['Destination Lat'], train_with_rider_info['Destination Long'])


In [379]:
test_with_rider_info['Bearing'] = bearing_test
train_with_rider_info['Bearing'] = bearing_train


In [380]:
# drop redundant columns train data
train_with_rider_info.drop(columns=['Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
                                    'Arrival at Pickup - Day of Month','Arrival at Pickup - Weekday (Mo = 1)',
                                    'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)',
                                    'Arrival at Destination - Day of Month',
                                    'Arrival at Destination - Weekday (Mo = 1)',
                                    'Arrival at Destination - Time',
                                    'Pickup Lat', 'Pickup Long',
                                    'Destination Lat', 'Destination Long',
                                    'Vehicle Type', 'Order No',
                                    'User Id', 'Rider Id',
                                    'Precipitation in millimeters'], inplace=True)


In [381]:
# drop redundant columns test data
test_with_rider_info.drop(columns=['Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
                                   'Arrival at Pickup - Day of Month', 'Arrival at Pickup - Weekday (Mo = 1)',
                                   'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)',
                                   'Vehicle Type', 'User Id', 'Rider Id', 'Precipitation in millimeters',
                                   'Pickup Lat', 'Pickup Long',
                                   'Destination Lat', 'Destination Long'], inplace=True)


In [382]:
train_with_rider_info.rename(columns={1: "Platform Type 1", 2: "Platform Type 2",
                                      3: "Platform Type 3", 4: "Platform Type 4"}, inplace=True)

test_with_rider_info.rename(columns={1: "Platform Type 1", 2: "Platform Type 2",
                                     3: "Platform Type 3", 4: "Platform Type 4"}, inplace=True)


In [383]:
train_with_rider_info.rename(columns={'Placement - Day of Month': 'Day of Month',
                                      'Placement - Weekday (Mo = 1)': 'Weekday (Mo = 1)'}, inplace=True)

test_with_rider_info.rename(columns={'Placement - Day of Month': 'Day of Month',
                                     'Placement - Weekday (Mo = 1)': 'Weekday (Mo = 1)'}, inplace=True)


In [384]:
# one hot encoding of the test_data['Weekday (Mo = 1)] column
test_with_rider_info['Weekday (Mo = 1)'] = test_with_rider_info['Weekday (Mo = 1)'].astype('category')
test_with_rider_info = pd.concat([test_with_rider_info.drop(columns=['Weekday (Mo = 1)']), pd.get_dummies(test_with_rider_info['Weekday (Mo = 1)'])], axis=1)


In [385]:
# one hot encoding of the train_data['Weekday (Mo = 1)] column
train_with_rider_info['Weekday (Mo = 1)'] = train_with_rider_info['Weekday (Mo = 1)'].astype('category')
train_with_rider_info = pd.concat([train_with_rider_info.drop(columns=['Weekday (Mo = 1)']), pd.get_dummies(train_with_rider_info['Weekday (Mo = 1)'])], axis=1)


In [386]:

train_with_rider_info.rename(columns={1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 
                                      4: 'Thursday', 5: 'Friday', 6: 'Saturday',
                                      7: 'Sunday'
                                      }, inplace=True)

test_with_rider_info.rename(columns={1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 
                                      4: 'Thursday', 5: 'Friday', 6: 'Saturday',
                                      7: 'Sunday'}, inplace=True)


In [256]:
# labeling part of the day in train data 
# train_with_rider_info['Placement - Time'] = train_with_rider_info['Placement - Time'].apply(time_to_day_part)
# train_with_rider_info['Confirmation - Time'] = train_with_rider_info['Confirmation - Time'].apply(time_to_day_part)
# train_with_rider_info['Arrival at Pickup - Time'] = train_with_rider_info['Arrival at Pickup - Time'].apply(time_to_day_part)
# train_with_rider_info['Pickup - Time'] = train_with_rider_info['Pickup - Time'].apply(time_to_day_part)


In [257]:
# labeling part of the day in test data
# test_with_rider_info['Placement - Time'] = test_with_rider_info['Placement - Time'].apply(time_to_day_part)
# test_with_rider_info['Confirmation - Time'] = test_with_rider_info['Confirmation - Time'].apply(time_to_day_part)
# test_with_rider_info['Arrival at Pickup - Time'] = test_with_rider_info['Arrival at Pickup - Time'].apply(time_to_day_part)
# test_with_rider_info['Pickup - Time'] = test_with_rider_info['Pickup - Time'].apply(time_to_day_part)


In [413]:
# label encoding, one hot encoding, and renaming of times
# labelencoder = LabelEncoder()
# train_with_rider_info['Placement - Time'] = labelencoder.fit_transform(train_with_rider_info['Placement - Time'])
# train_with_rider_info['Placement - Time'] = train_with_rider_info['Placement - Time'].astype('category')
# train_with_rider_info = pd.concat([train_with_rider_info.drop(columns=['Placement - Time']), pd.get_dummies(train_with_rider_info['Placement - Time'])], axis=1)



In [388]:
X = train_with_rider_info.drop(columns='Time from Pickup to Arrival')
Y = train_with_rider_info['Time from Pickup to Arrival']


In [389]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


In [411]:
regressor = RandomForestRegressor(n_estimators=228, 
                                  max_depth=100,
                                  max_features=4, min_samples_leaf=4,
                                  min_samples_split=8,
                                  random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)


In [412]:
mean_squared_error(y_test/60, y_pred/60)


174.9888379758989

In [406]:
grid_param = {
    'n_estimators': list(range(220,230,2)),
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3, 4],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
}


In [407]:
gd_sr = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid=grid_param,
    scoring='neg_mean_absolute_error',
    cv=3,
    n_jobs=-1)


In [408]:
gd_sr.fit(X_train, y_train)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [220, 222, 224, 226, 228], 'max_depth': [80, 90, 100, 110], 'max_features': [2, 3, 4], 'min_samples_leaf': [3, 4, 5], 'min_samples_split': [8, 10, 12]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [409]:
best_parameters = gd_sr.best_params_


In [410]:
best_parameters

{'max_depth': 100,
 'max_features': 4,
 'min_samples_leaf': 4,
 'min_samples_split': 8,
 'n_estimators': 228}

In [357]:
final_predict = regressor.predict(test_with_rider_info.drop(columns='Order No'))
test_with_rider_info['Time from Pickup to Arrival'] = final_predict



In [ ]:
submission = test_with_rider_info[['Order No','Time from Pickup to Arrival' ]]
submission['Time from Pickup to Arrival'] = submission['Time from Pickup to Arrival'].astype(int)


In [367]:
submission.to_csv('/Users/rade_dragosavac/PycharmProjects/SendyLogisticsChallenge /Data/Submission.csv', index=False)
